# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
cities_df = pd.read_csv('../WeatherPy/cities_df2.csv')
cities_df

,Unnamed: 0,city_name,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Rikitea,-23.12,-134.97,70.56,77,34,11.18,PF,1596687558
1,1,Puerto Ayora,-0.74,-90.35,66.00,96,81,3.00,EC,1596687558
2,2,Tura,25.52,90.22,77.07,96,99,5.73,IN,1596687559
3,3,Ushuaia,-54.80,-68.30,37.40,74,40,11.41,AR,1596687559
4,4,Cape Town,-33.93,18.42,50.00,93,75,4.70,ZA,1596687442
...,...,...,...,...,...,...,...,...,...,...
551,551,Esso,55.93,158.70,61.50,44,93,10.16,RU,1596687782
552,552,Kunming,25.04,102.72,82.00,64,75,2.24,CN,1596687783
553,553,Dahra,36.25,0.85,77.00,36,0,3.36,DZ,1596687783
554,554,Belz,47.68,-3.17,62.01,87,5,3.27,FR,1596687784


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:


# Store latitude and longitude in locations
locations = cities_df[['latitude', 'longitude']]

# Fill NaN values and convert to float
# rating = airport_df["Airport Rating"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
humidity = cities_df['humidity'].astype(int)
heat_layer = gmaps.heatmap_layer(locations, weights=(1.5 * humidity), 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#humidity less than 40
#temperature less than 75
#cloudiness less than 20
ideal_loc_df = cities_df.loc[(cities_df['humidity'] < 40) & (cities_df['max_temp'] < 75) & (cities_df['cloudiness'] < 20)]
ideal_loc_df

,Unnamed: 0,city_name,latitude,longitude,max_temp,humidity,cloudiness,wind_speed,country,date
25,25,Mayor Pablo Lagerenza,-19.93,-60.77,73.58,37,0,6.44,PY,1596687570
65,65,Araguaçu,-12.93,-49.83,68.50,31,0,8.34,BR,1596687585
117,117,Clarence Town,-32.58,151.78,60.01,30,0,11.41,AU,1596687035
174,174,Luau,-10.71,22.22,56.93,35,0,5.23,AO,1596687625
493,493,Flinders,-34.58,150.86,62.01,35,0,1.99,AU,1596687761
523,523,Port Macquarie,-31.43,152.92,63.00,39,0,1.01,AU,1596687772


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = ideal_loc_df
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() + ['Hotel Name'])

In [42]:
#create dataframe of just latlongs
lat_longs_df = (hotel_df[['latitude', 'longitude']])
lat_longs_df

,latitude,longitude
25,-19.93,-60.77
65,-12.93,-49.83
117,-32.58,151.78
174,-10.71,22.22
493,-34.58,150.86
523,-31.43,152.92


In [43]:
for index, row in lat_longs_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{row[0]}, {row[1]}",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        pass

In [44]:
#drop missing values
hotel_df = hotel_df.dropna()

In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [39]:
# Add marker layer ontop of heat map

markers= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))